In [5]:
#Image preprocessing aims at modifying the image in order to extract the representative features for training models
#reference of method: http://francescopochetti.com/text-recognition-natural-scenes/#first
#Computer font dataset from Chars74k dataset
#source: http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/
#Packages numpy and skimage required
import os
import numpy as np
from skimage.feature import hog
from skimage import color
from skimage.io import imread
from skimage.transform import resize

#read_images_fnt function read font character images in src file and extract the features through Histrogram of Oriented Gradient
def read_images_fnt(path):

    count=0
    features=[]
    for imagename in os.listdir(path):
        #Change image from RGB color image to a grayscale image
        image = color.rgb2gray(imread(path+"/"+imagename))
        #Resize image to 32x32 pixel
        imageResized = resize( image, (32,32) )
        #Extract features through Histogram of Oriented Gradient and create a list of features
        #reference: http://scikit-image.org/docs/dev/auto_examples/features_detection/plot_hog.html
        #1. (optional) global image normalisation
        #2. computing the gradient image in x and y
        #3. computing gradient histograms
        #4. normalising across blocks
        #5. flattening into a feature vector
        fd = hog(imageResized, cells_per_block=(1, 1))
        features.append(fd.tolist())
        count=count+1
    return features,count

In [ ]:
#62 classes including 0-9, A-Z and a-z
list_of_label=["0","1","2","3","4","5","6","7","8","9",
               "A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z",
               "a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]
#Dataset from English words generator
#Preprocess training set of 13144 character font images from English words generator
y_gen=[]
for char in list_of_label:
    y_gen.extend([char]*4)
y_gen=y_gen*53
X_gen=[]
X_gen_new,temp=read_images_fnt("src/words_generated")
X_gen.extend(X_gen_new)

In [6]:
#Packages numpy and skimage required
import numpy as np

from skimage.feature import hog
from skimage import color, exposure
from skimage.io import imread
from skimage.transform import resize
from skimage.filters.rank import median
from skimage.morphology import disk

#read_images function read character images in src file and extract the features through Histrogram of Oriented Gradient
def read_images(path,type,n):
    features=[]
    for i in range(1,n+1):
        #Change image from RGB color image to a grayscale image
        image = color.rgb2gray(imread(path+"/"+str(i)+"."+type))
        #Apply median filter to denoise image
        image_denoise=median(image, disk(2))
        #Stretch contrast by rescaling intensity
        p2, p98 = np.percentile(image_denoise, (2, 98))
        image_contrast = exposure.rescale_intensity(image_denoise, in_range=(p2, p98))
        #Resize image to 32x32 pixel
        image_resized = resize( image_contrast, (32,32) )
        #Extract features through Histogram of Oriented Gradient and create a list of features
        #http://scikit-image.org/docs/dev/auto_examples/features_detection/plot_hog.html
        fd = hog(image_resized, cells_per_block=(1, 1))
        features.append(fd.tolist())
    return features

#Subset of street view images from Chars 74k dataset
#source: https://www.kaggle.com/c/street-view-getting-started-with-julia/data
#Preprocess training set of 6283 character images in street view
X_char=[]
X_char=read_images("src/train_char","bmp",6283)


#Object Recognition dataset
#source: https://www.kaggle.com/c/cifar-10/data
#Preprocess training set of 14000 non-text object images
X_obj=[]
X_obj=read_images("src/train_obj","png",14000)



C:\Users\AndyTam\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\AndyTam\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [9]:
import pickle

#Save preprocessed features for training set of street view character and non-text object images
with open("X_char.txt", "wb") as X_file_w:   #Pickling
       pickle.dump(X_char, X_file_w)
with open("X_obj.txt", "wb") as X_obj_file_w:   #Pickling
       pickle.dump(X_obj, X_obj_file_w)
with open("X_gen.txt", "wb") as X_gen_file_w:   #Pickling
       pickle.dump(X_gen, X_gen_file_w)
with open("y_gen.txt", "wb") as y_gen_file_w:   #Pickling
       pickle.dump(y_gen, y_gen_file_w)

In [2]:
import scipy.io
#EMNIST handwritten character dataset
#source: https://www.nist.gov/itl/iad/image-group/emnist-dataset
#reference of reading the dataset: https://github.com/Coopss/EMNIST/blob/master/training.py
#Load dataset in mat format
mat = scipy.io.loadmat('src/emnist-byclass.mat')
max_ = len(mat['dataset'][0][0][0][0][0][0])
#Load images in 28x28 pixels
training_images = mat['dataset'][0][0][0][0][0][0][:max_].reshape(max_, 28,28)
training_labels = mat['dataset'][0][0][0][0][0][1][:max_].reshape(max_)
y_hand=training_labels.tolist()

In [14]:
#Package skimage required
from skimage.transform import resize
from skimage.feature import hog
from skimage import color
training_images_resize=[]
X_hand=[]
for i in range(0,max_):
    #Extract features through Histogram of Oriented Gradient and create a list of features
    #http://scikit-image.org/docs/dev/auto_examples/features_detection/plot_hog.html
    fd = hog(resize(training_images[i], (32,32) ), cells_per_block=(1, 1))
    X_hand.append(fd.tolist())

C:\Users\AndyTam\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\AndyTam\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [23]:
#Save features for training set of handwritten character images
with open("X_hand.txt", "wb") as X_hand_file_w:   #Pickling
       pickle.dump(X_hand, X_hand_file_w)
with open("y_hand.txt", "wb") as y_hand_file_w:   #Pickling
       pickle.dump(y_hand, y_hand_file_w)